# AI transpiler passes

The AI-powered transpiler passes are passes that work as a drop-in replacement of "traditional" Qiskit passes for some transpiling tasks. They often produce better results than existing heuristic algorithms (such as lower depth and CNOT count), but are also much faster than optimization algorithms such as Boolean satisfiability solvers. The AI transpiler passes can run on your local environment or on the cloud using the Qiskit Transpiler Service if you are part of the IBM Quantum&reg; Premium Plan.


<Admonition type="note">
    The AI-powered transpiler passes are in beta release status, subject to change.
    If you have feedback or want to contact the developer team, please use this [Qiskit Slack Workspace channel](https://qiskit.slack.com/archives/C06KF8YHUAU).
</Admonition>

The following passes are currently available:

**Routing passes**
 - `AIRouting`: Layout selection and circuit routing

**Circuit synthesis passes**
 - `AICliffordSynthesis`: Clifford circuit synthesis
 - `AILinearFunctionSynthesis`: Linear function circuit synthesis
 - `AIPermutationSynthesis`: Permutation circuit synthesis
 - `AIPauliNetworkSynthesis`: Pauli Network circuit synthesis

To use the AI transpiler passes, first [install the `qiskit-ibm-transpiler` package](/docs/guides/qiskit-transpiler-service#install-transpiler-service). Visit the [qiskit-ibm-transpiler API documentation](/docs/api/qiskit-ibm-transpiler) to get more information about the different options available.

## Run the AI transpiler passes locally or on the cloud

If you want to use the AI-powered transpiler passes in your local environment for free, install `qiskit-ibm-transpiler` with some extra dependencies as follows:

```sh
pip install qiskit-ibm-transpiler[ai-local-mode]
```

Without these extra dependencies, the AI-powered transpiler passes run on the cloud through the Qiskit Transpiler Service (available only for IBM Quantum Premium Plan users). After installing the extra dependencies, the default mode to run the AI-powered transpiler passes is to use your local machine.

## AI routing pass

The `AIRouting` pass acts both as a layout stage and a routing stage. It can be used within a `PassManager` as follows:

In [1]:
from qiskit.transpiler import PassManager
from qiskit.circuit.library import efficient_su2
from qiskit_ibm_transpiler.ai.routing import AIRouting
from qiskit_ibm_runtime import QiskitRuntimeService

backend = QiskitRuntimeService().backend("ibm_torino")
ai_passmanager = PassManager(
    [
        AIRouting(
            backend=backend,
            optimization_level=2,
            layout_mode="optimize",
            local_mode=True,
        )
    ]
)


circuit = efficient_su2(101, entanglement="circular", reps=1)

transpiled_circuit = ai_passmanager.run(circuit)

Here, the `backend` determines which coupling map to route for, the `optimization_level` (1, 2, or 3) determines the computational effort to spend in the process (higher usually gives better results but takes longer), and the `layout_mode` specifies how to handle the layout selection.
The `layout_mode` includes the following options:

- `keep`: This respects the layout set by the previous transpiler passes (or uses the trivial layout if not set). It is typically only used when the circuit must be run on specific qubits of the device. It often produces worse results because it has less room for optimization.
- `improve`: This uses the layout set by the previous transpiler passes as a starting point. It is useful when you have a good initial guess for the layout; for example, for circuits that are built in a way that approximately follows the device's coupling map. It is also useful if you want to try other specific layout passes combined with the `AIRouting` pass.
- `optimize`: This is the default mode. It works best for general circuits where you might not have good layout guesses. This mode ignores previous layout selections.
- `local_mode`: This flag determines where the `AIRouting` pass runs. If `False`, `AIRouting` runs remotely through the Qiskit Transpiler Service. If `True`, the package tries to run the pass in your local environment with a fallback to cloud mode if the required dependencies are not found.

## AI circuit synthesis passes

The AI circuit synthesis passes allow you to optimize pieces of different circuit types ([Clifford](/docs/api/qiskit/qiskit.quantum_info.Clifford), [Linear Function](/docs/api/qiskit/qiskit.circuit.library.LinearFunction), [Permutation](/docs/api/qiskit/qiskit.circuit.library.Permutation#permutation), Pauli Network) by re-synthesizing them. A typical way to use the synthesis pass is as follows:

In [ ]:
from qiskit.transpiler import PassManager

from qiskit_ibm_transpiler.ai.routing import AIRouting
from qiskit_ibm_transpiler.ai.synthesis import AILinearFunctionSynthesis
from qiskit_ibm_transpiler.ai.collection import CollectLinearFunctions
from qiskit_ibm_transpiler.ai.synthesis import AIPauliNetworkSynthesis
from qiskit_ibm_transpiler.ai.collection import CollectPauliNetworks
from qiskit.circuit.library import efficient_su2

ibm_torino = QiskitRuntimeService().backend("ibm_torino")
ai_passmanager = PassManager(
    [
        AIRouting(
            backend=ibm_torino,
            optimization_level=3,
            layout_mode="optimize",
            local_mode=True,
        ),  # Route circuit
        CollectLinearFunctions(),  # Collect Linear Function blocks
        AILinearFunctionSynthesis(
            backend=ibm_torino, local_mode=True
        ),  # Re-synthesize Linear Function blocks
        CollectPauliNetworks(),  # Collect Pauli Networks blocks
        AIPauliNetworkSynthesis(
            backend=ibm_torino, local_mode=True
        ),  # Re-synthesize Pauli Network blocks.
    ]
)

circuit = efficient_su2(10, entanglement="full", reps=1)

transpiled_circuit = ai_passmanager.run(circuit)

The synthesis respects the coupling map of the device: it can be run safely after other routing passes without disturbing the circuit, so the overall circuit will still follow the device restrictions. By default, the synthesis will replace the original sub-circuit only if the synthesized sub-circuit improves the original (currently only checking CNOT count), but this can be forced to always replace the circuit by setting `replace_only_if_better=False`.

The following synthesis passes are available from `qiskit_ibm_transpiler.ai.synthesis`:

- *AICliffordSynthesis*: Synthesis for [Clifford](/docs/api/qiskit/qiskit.quantum_info.Clifford) circuits (blocks of `H`, `S`, and `CX` gates). Currently up to nine qubit blocks.
- *AILinearFunctionSynthesis*: Synthesis for [Linear Function](/docs/api/qiskit/qiskit.circuit.library.LinearFunction) circuits (blocks of `CX` and `SWAP` gates). Currently up to nine qubit blocks.
- *AIPermutationSynthesis*: Synthesis for [Permutation](/docs/api/qiskit/qiskit.circuit.library.Permutation#permutation) circuits (blocks of `SWAP` gates). Currently available for 65, 33, and 27 qubit blocks.
- *AIPauliNetworkSynthesis*: Synthesis for Pauli Network circuits (blocks of `H`, `S`, `SX`, `CX`, `RX`, `RY` and `RZ` gates). Currently up to six qubit blocks.

We expect to gradually increase the size of the supported blocks.

All passes use a thread pool to send several requests in parallel. By default, the number for max threads is the number of cores plus four (default values for the `ThreadPoolExecutor` Python object). However, you can set your own value with the `max_threads` argument at pass instantiation. For example, the following line instantiates the `AILinearFunctionSynthesis` pass, which allows it to use a maximum of 20 threads.

```python
AILinearFunctionSynthesis(backend=ibm_torino, max_threads=20)  # Re-synthesize Linear Function blocks using 20 threads max
```

You can also set the environment variable `AI_TRANSPILER_MAX_THREADS` to the desired number of maximum threads, and all synthesis passes instantiated after that will use that value.

For the AI synthesis passes to synthesize a sub-circuit, it must lay on a connected subgraph of the coupling map (one way to do this is with a routing pass before collecting the blocks, but this is not the only way to do it). The synthesis passes will automatically check that the specific subgraph is supported, and if not, it will raise a warning and leave the original sub-circuit unchanged.

The following custom collection passes for Cliffords, Linear Functions and Permutations that can be imported from `qiskit_ibm_transpiler.ai.collection` also complement the synthesis passes:

- *CollectCliffords*: Collects Clifford blocks as `Instruction` objects and stores the original sub-circuit to compare against it after synthesis.
- *CollectLinearFunctions*: Collects blocks of `SWAP` and `CX` as `LinearFunction` objects and stores the original sub-circuit to compare against it after synthesis.
- *CollectPermutations*: Collects blocks of `SWAP` circuits as `Permutations`.
- *CollectPauliNetworks*: Collects Pauli Network blocks and stores the original sub-circuit to compare against it after synthesis.

These custom collection passes limit the sizes of the collected sub-circuits so they are supported by the AI-powered synthesis passes. Therefore, it is recommended to use them after the routing passes and before the synthesis passes for a better overall optimization.

## Hybrid heuristic-AI circuit transpilation

The `qiskit-ibm-transpiler` allows you to configure a hybrid pass manager that combines the best of Qiskit's heuristic and the AI-powered transpiler passes. This feature behaves similarly to the Qiskit `generate_pass_manager` method. A typical way to use `generate_ai_pass_manager` is as follows:

In [ ]:
from qiskit_ibm_transpiler import generate_ai_pass_manager
from qiskit.circuit.library import efficient_su2
from qiskit_ibm_runtime import QiskitRuntimeService


backend = QiskitRuntimeService().backend("ibm_torino")
torino_coupling_map = backend.coupling_map


su2_circuit = efficient_su2(101, entanglement="circular", reps=1)

ai_transpiler_pass_manager = generate_ai_pass_manager(
    coupling_map=torino_coupling_map,
    ai_optimization_level=3,
    optimization_level=3,
    ai_layout_mode="optimize",
)

ai_su2_transpiled_circuit = ai_transpiler_pass_manager.run(su2_circuit)

The following options are used in this example:
* `coupling_map` - Specifies which coupling map to use for the transpilation.
* `ai_optimization_level` - Specifies the level of optimization (1-3) to use for the AI components of the PassManager.
* `optimization_level` - Specifies how much optimization to perform on the circuit for the heuristic components of the PassManager.
* `ai_layout_mode` - Specifies how the AI routing part of the PassManager handles the layout. Refer to the [AI routing pass](#ai-routing-pass) section to review the configuration options for this `ai_layout_mode` parameter.

## Limits

Refer to the [Qiskit Transpiler Service documentation](./qiskit-transpiler-service#limits-of-the-qiskit-transpiler-service) for more information about the limits that apply to the AI-powered transpiler passes.

## Citation

If you use any AI-powered feature from the Qiskit Transpiler Service in your research, use [the recommended citation](./qiskit-transpiler-service#citation).